In [1]:
import lmql
import asyncio
import os
from dotenv import load_dotenv
load_dotenv('./env.txt')

True

# Simple token constraint

In [2]:
@lmql.query
async def tipo_test():
    '''lmql
    argmax
        "Pregunta: Sevilla es una ciudad ubicada en que parte de Espana \n"
        "A) Sur \n"
        "B) Norte \n"
        "C) Este \n"
        "D) Afuera de Espana \n"
        "Respuesta: [RESPUESTA]"
    from
        "openai/text-davinci-003"
    where
        RESPUESTA in set(["A", "B", "C", "D"])
    '''

In [3]:
r = await tipo_test()

In [4]:
r

LMQLResult(prompt='Pregunta: Sevilla es una ciudad ubicada en que parte de Espana \nA) Sur \nB) Norte \nC) Este \nD) Afuera de Espana \nRespuesta: A', variables={'RESPUESTA': 'A'}, distribution_variable=None, distribution_values=None)

In [6]:
r[0].variables

{'RESPUESTA': 'A'}

# Distribution

In [7]:
# Este parece no funcionar bien con OpenAI api

@lmql.query
async def sentiment_analysis(input):
    '''lmql
    argmax
        "Classifica el sentimiento de la siguiente frase como POS, NEG O NEUT: \n"
        "{input} \n"
        "El sentimento de la frase es [SENTIMENT]"
    from
        "openai/text-davinci-003"
    distribution
        SENTIMENT in ["POS", "NEUT", "NEG"]
    '''

In [8]:
r = await sentiment_analysis('Estoy muy feliz hoy!')

/usr/local/lib/python3.10/dist-packages/lmql/runtime/openai_integration.py:248: UserWarning: warning: The OpenAI API has merged 1 token(s) server-side, which will reflect in inaccurate 0.0 scores in the decoding tree
  warnings.warn("warning: The OpenAI API has merged {} token(s) server-side, which will reflect in inaccurate 0.0 scores in the decoding tree".format(server_side_swallowed_tokens))


In [9]:
r.variables.keys()

dict_keys(['SENTIMENT', 'P(SENTIMENT)', 'log P(SENTIMENT)'])

In [10]:
r.variables.get('P(SENTIMENT)')

[('POS', 0.3336377781463958),
 ('NEUT', 0.33275176110373544),
 ('NEG', 0.33361046074986866)]

# JSON response

In [11]:
from dataclasses import dataclass

In [12]:
# Only works in playground :p ?? 
# This only works if install directly from github, not pip (ie pip install git+https://github.com/eth-sri/lmql)


@lmql.query
async def jsonize():
    '''lmql
        from dataclasses import dataclass
        import lmql
        
        @dataclass
        class Event:
            nombre: str
            descripcion: str
                
        @dataclass
        class Meetup:
            nombre: str
            organizadores: str
            event: Event
    
        argmax
            """
            Por favor, coloca el texto a continuación en un formato JSON estructurado:
            IA Generativa Sevilla es un grupo de meetup colocada en Sevilla, Espana. Los organizadores son Ivan y Mathias
            El evento de hoy trata de dominando un LLM utilizando LMQL
            structured: [MEETUP]
            """
        from
            "openai/text-davinci-003"
        where
            type(MEETUP) is Meetup
    '''

In [13]:
r = await jsonize()
r

LMQLResult(prompt="\nPor favor, coloca el texto a continuación en un formato JSON estructurado:\nIA Generativa Sevilla es un grupo de meetup colocada en Sevilla, Espana. Los organizadores son Ivan y Mathias\nEl evento de hoy trata de dominando un LLM utilizando LMQL\nstructured: Meetup(nombre='IA Generativa Sevilla', organizadores='Ivan y Mathias', event=Event(nombre='Dominando un LLM utilizando LMQL', descripcion='Un grupo de meetup en Sevilla, España'))\n", variables={'MEETUP': Meetup(nombre='IA Generativa Sevilla', organizadores='Ivan y Mathias', event=Event(nombre='Dominando un LLM utilizando LMQL', descripcion='Un grupo de meetup en Sevilla, España'))}, distribution_variable=None, distribution_values=None)

In [14]:
r.variables['MEETUP']

Meetup(nombre='IA Generativa Sevilla', organizadores='Ivan y Mathias', event=Event(nombre='Dominando un LLM utilizando LMQL', descripcion='Un grupo de meetup en Sevilla, España'))

# Create json without dataclasses

In [15]:
@lmql.query
async def jsonize():
    '''lmql
    argmax 
        """
        Por favor, coloca el texto a continuación en un formato JSON estructurado:

        IA Generativa Sevilla es un grupo de meetup colocada en Sevilla, Espana. Los organizadores son Ivan y Mathias
        El evento de hoy trata de dominando un LLM utilizando LMQL

        {{
        "meetup-name": "[NAME]",
        "location": "[LOCATION]",
        "organizers": "[ORGANIZERS]",
        "description": "[DESCRIPTION]",
        }}
        """
    from
        "openai/text-davinci-003" 
    where
        STOPS_BEFORE(NAME, '"') and STOPS_BEFORE(DESCRIPTION, '"') and STOPS_BEFORE(LOCATION, '"') and STOPS_BEFORE(ORGANIZERS, '"') and len(NAME) < 25
    '''

In [16]:
r = await jsonize()
r

/usr/local/lib/python3.10/dist-packages/lmql/runtime/bopenai/batched_openai.py:741: OpenAILogitBiasLimitationWarning: the required logit_bias is too large to be handled by the OpenAI API and will be limited to the first 300 tokens. This can lead to the violation of the provided constraints or undesired model output. To avoid this use less broad or no constraints.
  warnings.warn("the required logit_bias is too large to be handled by the OpenAI API and will be limited to the first 300 tokens. This can lead to the violation of the provided constraints or undesired model output. To avoid this use less broad or no constraints.", category=OpenAILogitBiasLimitationWarning)


LMQLResult(prompt='\nPor favor, coloca el texto a continuación en un formato JSON estructurado:\nIA Generativa Sevilla es un grupo de meetup colocada en Sevilla, Espana. Los organizadores son Ivan y Mathias\nEl evento de hoy trata de dominando un LLM utilizando LMQL\n{\n"meetup-name": "IA Generativa Sevilla",\n"location": "Sevilla, Espana",\n"organizers": "Ivan y Mathias",\n"description": "Dominando un LLM utilizando LMQL",\n}\n', variables={'NAME': 'IA Generativa Sevilla', 'LOCATION': 'Sevilla, Espana', 'ORGANIZERS': 'Ivan y Mathias', 'DESCRIPTION': 'Dominando un LLM utilizando LMQL'}, distribution_variable=None, distribution_values=None)

In [17]:
r[0].variables

{'NAME': 'IA Generativa Sevilla',
 'LOCATION': 'Sevilla, Espana',
 'ORGANIZERS': 'Ivan y Mathias',
 'DESCRIPTION': 'Dominando un LLM utilizando LMQL'}

# Calling function / control flow

In [5]:
import requests

In [6]:
url = "https://api.dictionaryapi.dev/api/v2/entries/en/{word}"

In [7]:
async def get_definitions(word):
    data = requests.get(url.format(word=word)).json()
    # Extract the definitions
    definitions = []
    for meaning in data[0]['meanings'][0]['definitions']:
        definitions.append(f"definition: {meaning['definition']}")
    return definitions

In [8]:
@lmql.query
async def example_sentence(term):
    '''lmql
    argmax
        """
        Please provide an example sentences for each definition of the word: {term}
        """

        definitions = await get_definitions(term)
        definition_strings = "\n".join(definitions)

        """
        Here are the dictionary definitions:
        {definition_strings}
        """
        
        examples = []
        for i in range(len(definitions)):
            "-[EXAMPLE]" where STOPS_AT(EXAMPLE, "\n")
            examples.append(EXAMPLE.strip())
        return examples
    from
        "openai/text-davinci-003"
    '''
    

In [9]:
response = await example_sentence("Asynchronous")

In [10]:
response

['Example sentence 1: The asynchronous nature of the internet allows for multiple users to access the same website simultaneously.',
 'Example sentence 2: The asynchronous request allowed the client to continue browsing while the server processed the data.',
 'Example sentence 3: The asynchronous communication between the two computers allowed for multiple actions to occur at the same time.']

# Open Source Model

In [ ]:
# hay que crear un server local:
# https://docs.lmql.ai/en/stable/language/hf.html

In [48]:
@lmql.query
async def example_os_model_sql(pregunta):
        '''lmql
        argmax 
            """
            Dada la tabla a continuación, por favor responda la pregunta a continuación con una consulta SQL válida.
            Por favor, utilice solamente la tabla que se encuentra a continuación:

            ### Tablas:
            CREATE TABLE transactions (
                transaction_id INT PRIMARY KEY AUTO_INCREMENT,
                customer_id INT NOT NULL,
                product_id INT NOT NULL,
                transaction_date DATE NOT NULL,
                amount DECIMAL(10, 2) NOT NULL,
                payment_method VARCHAR(50) NOT NULL,
                transaction_status VARCHAR(20) NOT NULL,
                FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
                FOREIGN KEY (product_id) REFERENCES products(product_id)
            );
    
            Q: {pregunta}
            A: SELECT [QUERY]
            """
        from
            lmql.model("meta-llama/Llama-2-7b-chat-hf", endpoint="localhost:9999", tokenizer='hf-internal-testing/llama-tokenizer', device=0)
        where
            STOPS_BEFORE(QUERY, ';')
        '''

In [49]:
resp = await example_os_model_sql(pregunta="Por favor, proporciona las últimas cinco transacciones.")

In [59]:
print('SELECT' + resp.variables.get('QUERY'))

SELECT
    *
FROM 
    transactions

ORDER BY 
    transaction_date DESC

LIMIT 5


# Distribucion con open source model

In [56]:
# Este parece no funcionar bien con OpenAI api

@lmql.query
async def sentiment_analysis(input):
    '''lmql
    argmax
        "Classifica el sentimiento de la siguiente frase como positivo, negativo o neutral: \n"
        "###Frase \n"
        "{input} \n"
        "Pensamiento: [REASONING] \n"
        "Por eso creo que el sentimento de la frase es [SENTIMENT]"
    from
        lmql.model("meta-llama/Llama-2-7b-chat-hf", endpoint="localhost:9999", tokenizer='hf-internal-testing/llama-tokenizer', device=0)
    distribution
        SENTIMENT in ["POS", "NEUT", "NEG"]
    '''

In [57]:
r = await sentiment_analysis('Estoy muy feliz hoy!')

In [58]:
r.variables

{'REASONING': '\nLa frase "Estoy muy feliz hoy" es una expresión positiva, ya que el término "feliz" tiene un significado que se refiere a un estado de ánimo alegre y satisfactorio. Por lo tanto, la frase se clasifica como positiva.',
 'SENTIMENT': 'POS',
 'P(SENTIMENT)': [('POS', 0.8116896641444746),
  ('NEUT', 0.15510277328034572),
  ('NEG', 0.03320756257517929)],
 'log P(SENTIMENT)': [('POS', -0.2086371988867235),
  ('NEUT', -1.8636673283641052),
  ('NEG', -3.40497764059052)]}